# 错误处理
1. python的错误是类, 所有的错误都继承于BaseException, 所以在捕获某类型的错误时, 会把该类型错误的子类也捕获.[常见的错误类型和继承关系](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
2. 如果错误没有被捕获, 他会一直往上跑, 直到最后被Python解释器捕获

```Python
# err.py:
def foo(s):
    return 10 / int(s)

def bar(s):
    return foo(s) * 2

def main():
    bar('0')

main()
```

    执行结果如下:
    ```Python
    $ python3 err.py
    Traceback (most recent call last):
      File "err.py", line 11, in <module>
        main()
      File "err.py", line 9, in main
        bar('0')
      File "err.py", line 6, in bar
        return foo(s) * 2
      File "err.py", line 3, in foo
        return 10 / int(s)
    ZeroDivisionError: division by zero
    ```

3. 在bar()函数中，我们明明已经捕获了错误，但是，打印一个ValueError!后，又把错误通过raise语句抛出去了, 捕获错误目的只是记录一下，便于后续追踪。但是，由于当前函数不知道应该怎么处理该错误，所以，最恰当的方式是继续往上抛，让顶层调用者去处理。

```Python
# err_reraise.py

def foo(s):
    n = int(s)
    if n==0:
        raise ValueError('invalid value: %s' % s)
    return 10 / n

def bar():
    try:
        foo('0')
    except ValueError as e:
        print('ValueError!')
        raise

bar()
```

# unittest——单元测试框架
** 重要概念 **
- Test fixture(测试夹具)
- Test case(测试用例)
- Test suite(测试套件)
- Test runner(测试运行)
## 基本例子

```Python
import unittest

class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':
    unittest.main()
```

1. 通过继承unittest.Testcase来创建一个test case. 
2. 每个method都以test为前缀
3. self.assertEqual验证预期结果; self.assertTrue, self.assertFalse验证条件, self.assertRaises验证引发特定异常
4. TestCase.setUp, TestCase.tearDown允许在每个test method前和每个test method后执行相应指令
5. unittest.main()是运行测试的一种简单方法.
6. 通过python xx.py -v 加上-v选项会显示更详细的测试信息.

## Command-Line interface(命令行界面)
The unittest module can be used from command line to run tests from modules, classes or even individual test moduls.
```Python
python -m unittest test_module1 test_module2
python -m unittest test_module.TestClass
python -m unittest test_module.TestClass.test_method
```

当测试模块时, 要省略模块文件的.py后缀。```python -m unittest -v skip```<br \>
所有的命令行选项可通过```python -m unittest -h```查看

## Test discovery(测试发现)(待补全)
unittest支持简单的test discovery. 为了与test discovery兼容, 所有的测试文件必须是可以从项目的顶级目录导入的modeule或package. 
基本的命令行用法是: 
```
cd project_directory
python -m unittest discover
```

**作为一个捷径，python -m unittest 相当于 python -m unittest discover。如果要传递参数以进行测试发现，则必须显式地使用 discover 子命令。**

## 组织测试代码
- 单元测试的基本块是test case. 在unittest中, test case是unittest.TestCase的实例. 要创建自己的test case, 必须编写TestCase的子类或使用FunctionTestCase.
- TestCase的代码应该是entirly self contained, 这样他既可以自己运行, 或者搭配其他任意数量的TestCase运行.
- 最简单的TestCase子类包含简单的test method(method started with 'test')
- 一个TestCase实例中可以有多个test method, 他们可能会共用一些设置, 这些设置可以通过TestCase.setUp()来配置, TestCase.setUp在test method前运行(同理, TestCase.tearDown()在test method结束后运行)

```Python
import unittest

class WidgetTestCase(unittest.TestCase):
    def setUp(self):
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_widget_resize(self):
        self.widget.resize(100,150)
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')
```

**如果 setUp() 方法在测试运行时引发异常，那么框架将认为测试遇到错误，并且不会执行测试方法。**
<br \>
**If setUp() succeeded, tearDown() will be run whether the test method succeeded or not.**
<br \>
**Such a working environment for the testing code is called a fixture.**

- TestCase实例根据要测试的东西归为一组. unittest有一个收集要测试的内容的机制: test suite, 那就是unittest的TestSuite类. 在大多数情况下, 调用unittest.main()就可以自动实现, 但也可以自己设置test suit

```Python
def suite():
    suite = unittest.TestSuite()
    suite.addTest(WidgetTestCase('test_default_widget_size'))
    suite.addTest(WidgetTestCase('test_widget_resize'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())
```

- 你可以把test case和test suit放在要测试的代码中, 当更好的的方式是把test case和test suite作为测试代码, 单独放在一个module中:

    1. 测试代码本身可以从命令行单独运行
    2. 测试代码与要测试的代码可以很容易地区分开来
    3. 减少你为了通过测试, 而主动修改测试代码的可能
    4. 测试代码相对其他代码, 很少需要修改
    5. 分开后测试代码方便重构
    6. 写C的测试是要必须分开出来的, 形成一个好习惯

## Re-using old test code(待补全)
整合一些不适用unittest写的测试代码, 好像不是太常用

## Skipping test and expected failures
unittest支持skip test method or even classes. 此外, 他还支持标记'expected failture', 即遇见某个method或class会broke, 但并不把它当做failture. 
这些都可以通过skip装饰器很简单地实现.<br \>
**最基本的skip**
```Python
class MyTestCase(unittest.TestCase):

    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(mylib.__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass
```

<br \>
**skip 类**
```Python
@unittest.skip("showing class skipping")
class MySkippedTestCase(unittest.TestCase):
    def test_not_run(self):
        pass
```
<br \>
**expected failture**
```Python
class ExpectedFailureTestCase(unittest.TestCase):
    @unittest.expectedFailure
    def test_fail(self):
        self.assertEqual(1, 0, "broken")
```

### 常用skip装饰器
- @unittest.skip(reason), reason是字符串, 解释skip的原因
- @unittest.skipIf(condition, reason)
- @unittest.skipUnless(condition, reason)
这两个用法差不多, 都是condition为True时skip
- @unittest.expectedFailure

## Distinguishing test iterations using subtests
```Python
class NumbersTest(unittest.TestCase):

    def test_even(self):
        """
        Test that numbers between 0 and 5 are all even.
        """
        for i in range(0, 6):
            with self.subTest(i=i):
                self.assertEqual(i % 2, 0)
```

结果是:

```Python
======================================================================
FAIL: test_even (__main__.NumbersTest) (i=1)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "subtests.py", line 32, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

======================================================================
FAIL: test_even (__main__.NumbersTest) (i=3)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "subtests.py", line 32, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

======================================================================
FAIL: test_even (__main__.NumbersTest) (i=5)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "subtests.py", line 32, in test_even
    self.assertEqual(i % 2, 0)
AssertionError: 1 != 0

```

**如果不用subText, 将只会显示i=1时的一次错误**

# unittest的API
## TestCase
- setUp()
- tearDown()
- setUpClass()
- tearDownClass()
- assertEqual(first, second, msg=None)
- assertNotEqual(first, second, msg=None)
- assertTrue(expr, msg=None)
- assertFalse(expr, msg=None)
- assertIs(first, second, msg=None)
- assertIsNot(first, second, msg=None)
- assertIsNone(expr, msg=None)
- assertIsNotNone(expr, msg=None)
- assertIn(first, second, msg=None)
- assertNotIn(first, second, msg=None)
- assertIsInstance(obj, cls, msg=None)
- assertNotIsInstance(obj, cls, msg=None)
- assertRaises(exception, callable, *args, **kwds)
- assertRaises(exception, msg=None)
- assertRaisesRegex
- assertWarns
- assertWarnsRegex
- assertLogs(logger=None, level=None)
- assertAlmostEqual(first, second, places=7, msg=None, delta=None)
- assertNotAlmostEqual(first, second, places=7, msg=None, delta=None)
- assertGreater(first, second, msg=None)
- assertGreaterEqual(first, second, msg=None)
- assertLess(first, second, msg=None)
- assertLessEqual(first, second, msg=None)
- assertRegex(text, regex, msg=None)
- assertNotRegex(text, regex, msg=None)
- assertCountEqual(first, second, msg=None)
- addTypeEqualityFunc(typeobj, function)
- 太多了, 不列出来了. [查看文档](https://docs.python.org/3/library/unittest.html#module-unittest)


In [1]:
s = 'hello world'

In [3]:
s.isupper()

False

In [8]:
round(5.231232, 4)

5.2312